In [3]:
from utilities.postprocessing.postprocessing import * 
from utilities.postprocessing.main import *

In [29]:
from glob import glob

class TraversabilityDir():
    def __init__(self, root, maps_dir):
        self.root, self.maps_dir = root, maps_dir

    @property
    def meta(self):
        return pd.read_csv(self.bags_dir + '/meta.csv')

    @property
    def bags_dir(self):
        return path.normpath(self.root + '/bags')

    @property
    def csvs_dir(self):
        return path.normpath(self.root + '/csvs')

    @property
    def csvs_parsed_dir(self):
        return path.normpath(self.root + '/csvs_parsed')

    @property
    def csvs_patches_dir(self):
        return path.normpath(self.root + '/csvs_patches')

    @property
    def patches_dir(self):
        return path.normpath(self.root + '/patches')

    def should_convert2bag(self, filename):
        return path.exists(self.csvs + '/' + filename + '.csv')

    def exist(self, filename):
        return path.exists(self.csvs_parsed + '/' + filename + '.csv')
    

In [30]:
tr_dir = TraversabilityDir(root='/media/francesco/saetta/krock-dataset/val', maps_dir='./maps/val')

In [31]:
tr_dir.bags_dir

'/media/francesco/saetta/krock-dataset/val/bags'

In [120]:
from utilities.pipeline import MultiThreadWrapper

def hmread(path):
    hm = cv2.imread(path)
    hm = cv2.cvtColor(hm, cv2.COLOR_BGR2GRAY)
    return hm

class TraversabilityDataframe():
    def __init__(self, df, root):
        self.df = df
        self.root = root 
        
    @classmethod
    def from_dir(cls, dir, *args, **kwargs):
        return cls.from_meta(dir.meta, dir.csvs_parsed_dir, dir.maps_dir, dir.bags_dir,  *args, **kwargs)
    
    
    
    @classmethod
    def from_meta(cls, meta, df_dir, hm_dir, bags_dir=None, *args, **kwargs):
        names = meta['filename']
        maps = meta['map']
#         try:
#             df = pd.read_csv(df_dir + '/' + names[2] + '.csasdv')
#         except FileNotFoundError:
        hm = hmread(hm_dir + '/' + maps[2] + '.png')
        return cls.from_bag_path(path.normpath(bags_dir + '/' + names[2] + '.bag'), hm,  *args, **kwargs)

#         return MultiThreadWrapper(16, cls.from_bag, (names))
#         return cls.from_bag_path(path.normpath(bags_dir + '/' + names[0] + '.bag'), hm,  *args, **kwargs)
#         return cls(df, *args, **kwargs)

    @classmethod
    def from_bag_path(cls, bag_path, *args, **kwargs):
        df = rosbag_pandas.bag_to_dataframe(bag_path)
#         store
        return cls.from_bag_parsed(df, *args, **kwargs)
    
    @classmethod
    def from_bag_parsed(cls, df_path, hm, *args, **kwargs):
        df = pd.read_csv(df_dir + '/' + names[2] + '.csasdv')
        df = ParseDataframe()(df_path, hm)
        # get patches from bag converted 2 df
#         store
        return df
    
    @classmethod
    def from_df(cls):
        # get patches from
#         store
        pass


In [121]:
TraversabilityDataframe.from_dir(tr_dir, 0.66, root='/')

TypeError: __call__() takes 2 positional arguments but 3 were given

In [74]:
meta.iloc[0]

Unnamed: 0                     0
filename      slope_rocks3-1.0-0
map                 slope_rocks3
height                         1
Name: 0, dtype: object